# import modules & load files

import modules

In [53]:
import pandas as pd
import numpy as np
import math
import glob
import datetime
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: d:\myhub\Project1


load files

In [39]:
CTB9sp = pd.read_csv('./target_price_set/CTB9sp.csv', index_col=0)
CTB15sp = pd.read_csv('./target_price_set/CTB15sp.csv', index_col=0)
PTB9sp = pd.read_csv('./target_price_set/PTB9sp.csv', index_col=0)
PTB15sp = pd.read_csv('./target_price_set/PTB15sp.csv', index_col=0)

In [40]:
output = pd.read_csv("./result_quarter_output.csv", index_col=0)
labels = output.iloc[:,-7:]

In [41]:
labels

,VKOSPI_Label,LogisticRegression,DecisionTree,SVC,RandomForest,XGB,LGBM
2023-10-10,1.0,0.0,0.0,0.0,0.0,0,0.0
2023-10-11,1.0,0.0,0.0,0.0,0.0,0,0.0
2023-10-12,1.0,0.0,0.0,0.0,0.0,0,0.0
2023-10-13,0.0,0.0,0.0,0.0,0.0,0,0.0
2023-10-16,0.0,0.0,0.0,0.0,0.0,1,1.0
2023-10-17,0.0,1.0,1.0,1.0,1.0,0,0.0
2023-10-18,1.0,0.0,0.0,0.0,0.0,0,0.0
2023-10-19,0.0,0.0,0.0,0.0,0.0,0,0.0
2023-10-20,0.0,1.0,1.0,1.0,1.0,1,1.0
2023-10-23,1.0,1.0,0.0,1.0,1.0,1,1.0


In [42]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 2023-10-10 to 2023-11-03
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   VKOSPI_Label        19 non-null     float64
 1   LogisticRegression  19 non-null     float64
 2   DecisionTree        19 non-null     float64
 3   SVC                 19 non-null     float64
 4   RandomForest        19 non-null     float64
 5   XGB                 19 non-null     int64  
 6   LGBM                19 non-null     float64
dtypes: float64(6), int64(1)
memory usage: 1.2+ KB


In [43]:
labels.describe()

,VKOSPI_Label,LogisticRegression,DecisionTree,SVC,RandomForest,XGB,LGBM
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000
mean,0.526316,0.263158,0.368421,0.210526,0.263158,0.315789,0.263158
std,0.512989,0.452414,0.495595,0.418854,0.452414,0.477567,0.452414
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.500000,1.000000,0.000000,0.500000,1.000000,0.500000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [44]:
VKOSPI_Label = labels['VKOSPI_Label']
LogisticRegression = labels['LogisticRegression']
DecisionTree = labels['DecisionTree']
SVC = labels['SVC']
RandomForest = labels['RandomForest']
XGB = labels['XGB']
LGBM = labels['LGBM']

In [45]:
VKOSPI_Label.info()

<class 'pandas.core.series.Series'>
Index: 19 entries, 2023-10-10 to 2023-11-03
Series name: VKOSPI_Label
Non-Null Count  Dtype  
--------------  -----  
19 non-null     float64
dtypes: float64(1)
memory usage: 304.0+ bytes


매매전략별 수익률 계산
=====
if label == -1: # 하락 → 양매도
    인덱스(시간) 일치
    case 1: 스트래들매도
    CTB9sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매도함
    PTB9sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매도함
    CTB15sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매도(청산)함
    PTB15sp에서 인덱스(시간) 기준행사가 컬럼 값을 저장 매수(청산)함
    
    case 2: 스트랭글
    CTB9sp에서 인덱스(시간) 기준행사가+4 C매도
    PTB9sp에서 인덱스(시간) 기준행사가-4 P매도
    CTB15sp에서 인덱스(시간) 기준행사가+4 C매수
    PTB15sp에서 인덱스(시간) 기준행사가-4 P매수
    수익률 계산 = 매수 값 / 매도 값
    수익금 계산 = 매수 값 - 매도 값

    case 3: 버터플매수
        case 3-1:
            기준행사가 P매도 *2
            기준행사가-1 P매수 
            기준행사가+1 P매수
        case 3-2:
            기준행사가 C매도 *2
            기준행사가-1 C매수
            기준행사가+1 C매수
    
elif Label == 0: # 보합
    관망 how to code
    
elif label == 1: # 상승 → 양매수
    인덱스(시간) 일치
    case 1: 스트래들매수
    CTB9sp에서 인덱스(시간) 기준행사가 값을 저장 매수함
    PTB9sp에서 인덱스(시간) 기준행사가 값을 저장 매수함
    CTB15sp에서 인덱스(시간) 기준행사가 값을 저장 매도(청산)함
    PTB15sp에서 인덱스(시간) 기준행사가 값을 저장 매도(청산)함
    수익률 계산 = 매도 값 / 매수 값
    수익금 계산 = 매도 값 - 매수 값

    case 2: 스트랭글매수
    CTB9sp에서 인덱스(시간) 기준행사가+4 C매수
    PTB9sp에서 인덱스(시간) 기준행사가-4 P매수
    CTB15sp에서 인덱스(시간) 기준행사가+4 C매도
    PTB15sp에서 인덱스(시간) 기준행사가-4 P매도

    case 3: 버터플매수
        case 3-1:
            기준행사가 P매수 *2
            기준행사가-1 P매도
            기준행사가+1 P매도
        case 3-2:
            기준행사가 C매수 *2
            기준행사가-1 C매도
            기준행사가+1 C매도

실제 변동

In [64]:
VKOSPI_Label

2023-10-10    1.0
2023-10-11    1.0
2023-10-12    1.0
2023-10-13    0.0
2023-10-16    0.0
2023-10-17    0.0
2023-10-18    1.0
2023-10-19    0.0
2023-10-20    0.0
2023-10-23    1.0
2023-10-24    1.0
2023-10-25    0.0
2023-10-26    0.0
2023-10-27    1.0
2023-10-30    1.0
2023-10-31    1.0
2023-11-01    0.0
2023-11-02    1.0
2023-11-03    0.0
Name: VKOSPI_Label, dtype: float64

In [80]:
for i in range(len(VKOSPI_Label.index)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [82]:
VKOSPI_Label.loc[11]

KeyError: 11

In [94]:
for i in range(len(VKOSPI_Label.index)):
    d = VKOSPI_Label.index[i]
    if VKOSPI_Label.loc[d] == 0:
        try:
            VKOSPI_Label[['cost']] = CTB9sp['기준행사가'].loc[d]
            VKOSPI_Label[['cost']] += PTB9sp['기준행사가'].loc[d]
            print(VKOSPI_Label['cost'])
        except:
            print(np.nan)
        
    else:
        try:
            # VKOSPI_Label['cost'] += CTB9sp['기준행사가'].loc[i]
            # VKOSPI_Label['cost'] +=
            print(d, '1')    
        except:
            print(np.nan)

2023-10-10 1
2023-10-11 1
2023-10-12 1
10.75
-0.5
-3.53
2023-10-18 1
-3.24
5.050000000000001
2023-10-23 1
2023-10-24 1
nan
nan
2023-10-27 1
2023-10-30 1
2023-10-31 1
nan
2023-11-02 1
nan
cost 1


In [96]:
VKOSPI_Label

2023-10-10    1.00
2023-10-11    1.00
2023-10-12    1.00
2023-10-13    0.00
2023-10-16    0.00
2023-10-17    0.00
2023-10-18    1.00
2023-10-19    0.00
2023-10-20    0.00
2023-10-23    1.00
2023-10-24    1.00
2023-10-25    0.00
2023-10-26    0.00
2023-10-27    1.00
2023-10-30    1.00
2023-10-31    1.00
2023-11-01    0.00
2023-11-02    1.00
2023-11-03    0.00
cost          5.05
Name: VKOSPI_Label, dtype: float64